In [1]:
import pandas as pd
from sodapy import Socrata
from bs4 import BeautifulSoup
import requests
import numpy as np

from splinter import Browser
import time

In [23]:
client = Socrata("data.seattle.gov", None)
results = client.get("6vkj-f5xf", limit = 5000)
results_df = pd.DataFrame.from_records(results)

In [67]:
new = results_df['title'].str.split(" /",n=1,expand = True)
results_df["short title"]= new[0]
results_df.head()

,author,bibnum,floatingitem,isbn,itemcollection,itemcount,itemlocation,itemtype,publicationyear,publisher,reportdate,subjects,title,short title
0,"O'Ryan, Ellie",3011076,Floating,"1481425730, 1481425749, 9781481425735, 9781481...",ncrdr,1,qna,jcbk,2014.,"Simon Spotlight,",2017-09-01T00:00:00.000,"Musicians Fiction, Bullfighters Fiction, Best ...",A tale of two friends / adapted by Ellie O'Rya...,A tale of two friends
1,"Kishimoto, Masashi, 1974-",2248846,NA,1569319006,nycomic,1,lcy,acbk,"2003, c1999.","Viz,",2017-09-01T00:00:00.000,"Ninja Japan Comic books strips etc, Comic book...","Naruto. Vol. 1, Uzumaki Naruto / story and art...","Naruto. Vol. 1, Uzumaki Naruto"
2,"Scott, Jerry, 1955-",3209270,NA,"144945867X, 9781449458676",nycomic,1,bea,acbk,2014.,"Andrews McMeel Publishing,",2017-09-01T00:00:00.000,Duncan Jeremy Fictitious character Comic books...,"Peace, love & Wi-Fi : a ZITS treasury / by Jer...","Peace, love & Wi-Fi : a ZITS treasury"
3,"Carlile, Clancy, 1930-",1907265,NA,0786706155,cafic,1,cen,acbk,c1999.,"Carroll & Graf,",2017-09-01T00:00:00.000,"Hemingway Ernest 1899 1961 Fiction, Biographic...",The Paris pilgrims : a novel / Clancy Carlile.,The Paris pilgrims : a novel
4,NaN,1644616,NA,094020813X,canf,1,cen,acbk,"1991, c1988.","Red Alder Books/Down There Press,",2017-09-01T00:00:00.000,"Erotic literature American, American literatur...","Erotic by nature : a celebration of life, of l...","Erotic by nature : a celebration of life, of l..."


In [13]:
base_url = "https://en.wikipedia.org/wiki/The_New_York_Times_Fiction_Best_Sellers_of_"

year_range = np.arange(1971, 2018, dtype=int)
#year_range = [2018]
print(year_range)
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

[1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984
 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998
 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015 2016 2017]


In [14]:
# append to one data frame
year = base_url + str(1970)
tables = pd.read_html(year)
df_prime = tables[0]
new_header = df_prime.iloc[0] #grab the first row for the header
df_prime = df_prime[1:] #take the data less the header row
df_prime.columns = new_header #set the header row as the df header
df_prime = df_prime.dropna()
df_prime['Year'] = 1970

isbn =[]
for row in df_prime['Book']: 
        browser.visit(year)
        time.sleep(2)
        try:
            browser.click_link_by_partial_text(row)
            #time.sleep(2)
            browser.click_link_by_partial_href("Special:BookSources")
            html= browser.url
            isbn_num = html[50:]
            isbn.append(isbn_num)
            
        except:
            isbn.append("")
df_prime['ISBN']= isbn

In [70]:
("Evans,Scott".split(","))[:1]

['Evans']

In [88]:
type(str("Evans,Scott".split(",")[1:]) + str("Evans,Scott".split(",")[:1]))

str

In [15]:
for y in year_range:

    new_url = base_url + str(y)
    print(new_url)
    table1 = pd.read_html(new_url)
    df = table1[0]
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header
    df = df.dropna()
    df = df.drop_duplicates(['Book'], keep='first')
    df['Year'] = y
    isbn = []
    print(df['Book'])
    for row in df['Book']: 
        browser.visit(new_url)
        time.sleep(2)
        try:
            browser.click_link_by_partial_text(row)
            #time.sleep(2)
            browser.click_link_by_partial_href("Special:BookSources")
            html= browser.url
            isbn_num = html[50:]
            isbn.append(isbn_num)
            
        except:
            isbn.append("")
    df['ISBN']= isbn
    df_prime = df_prime.append(df, ignore_index = True)


https://en.wikipedia.org/wiki/The_New_York_Times_Fiction_Best_Sellers_of_1971
1                   Love Story
8                       QB VII
17    The Passions of the Mind
30                The Exorcist
42       The Day of the Jackal
45                      Wheels
Name: Book, dtype: object
https://en.wikipedia.org/wiki/The_New_York_Times_Fiction_Best_Sellers_of_1972
1                          Wheels
3                The Winds of War
20                       The Word
27    Jonathan Livingston Seagull
Name: Book, dtype: object
https://en.wikipedia.org/wiki/The_New_York_Times_Fiction_Best_Sellers_of_1973
1     Jonathan Livingston Seagull
12                The Odessa File
18             Once Is Not Enough
26         Breakfast of Champions
36               The Hollow Hills
47            The Honorary Consul
49                           Burr
Name: Book, dtype: object
https://en.wikipedia.org/wiki/The_New_York_Times_Fiction_Best_Sellers_of_1974
1                          Burr
18               W

https://en.wikipedia.org/wiki/The_New_York_Times_Fiction_Best_Sellers_of_1994
1                Slow Waltz in Cedar Bend
4                              Disclosure
9                                Accident
14                 The Celestine Prophecy
21                            Remember Me
24                            The Chamber
32                               The Gift
36                          Debt of Honor
42                                 Taltos
43                               Insomnia
50    Politically Correct Bedtime Stories
Name: Book, dtype: object
https://en.wikipedia.org/wiki/The_New_York_Times_Fiction_Best_Sellers_of_1995
1     Politically Correct Bedtime Stories
4                  The Celestine Prophecy
18                          The Rainmaker
26          The Bridges of Madison County
29                            Beach Music
31                      Memnoch the Devil
35                    From Potter's Field
38                     "L" Is for Lawless
40                  

https://en.wikipedia.org/wiki/The_New_York_Times_Fiction_Best_Sellers_of_2006
1                   Mary, Mary
3           "S" Is for Silence
4                  The Hostage
5            The Da Vinci Code
7                         Cell
10            The 5th Horseman
12                   The House
16                        Gone
17    Two Little Girls in Blue
21                  Beach Road
24                     At Risk
25                 The Husband
28                Twelve Sharp
31                 Angels Fall
32                     Phantom
34              Judge and Jury
38              Rise and Shine
39            The Book of Fate
40         The Thirteenth Tale
42            For One More Day
46               Lisey's Story
47                   Dear John
49                       Cross
Name: Book, dtype: object
https://en.wikipedia.org/wiki/The_New_York_Times_Fiction_Best_Sellers_of_2007
1             For One More Day
4                  Plum Lovin'
8              Step on a Crack
10          

https://en.wikipedia.org/wiki/The_New_York_Times_Fiction_Best_Sellers_of_2014
1                        Sycamore Row
4              The Invention of Wings
5                       The Goldfinch
9                        Private L.A.
10                 Concealed in Death
11                         The Chance
12                  Words of Radiance
13                       Night Broken
14                        Missing You
15                       Shadow Spell
16                           The King
17                   Carnal Curiosity
18                      The Collector
19                         The Target
21                         Unlucky 13
23                   The One and Only
24                          Skin Game
25                       Mr. Mercedes
26    Written in My Own Heart's Blood
27              Top Secret Twenty-One
28                          Invisible
30                         Act of War
31                   The Book of Life
32               Fifty Shades of Grey
33        

In [49]:
df_prime.head()

,Date,Book,Author,Year,ISBN,Author First,Author Last,ISBNx
0,January 4,The Godfather,Mario Puzo,1970,,Mario,Puzo,
1,February 8,The French Lieutenant's Woman,John Fowles,1970,0224616544,John,Fowles,0-224-61654-4
2,February 15,The Godfather,Mario Puzo,1970,,Mario,Puzo,
3,February 22,The French Lieutenant's Woman,John Fowles,1970,0224616544,John,Fowles,0-224-61654-4
4,May 10,Love Story,Erich Segal,1970,034012508X,Erich,Segal,0-340-12508-X


In [51]:
new_isbn= []
for x in df_prime['ISBN']:
    y = x.replace('-',"")
    new_isbn.append(y)
df_prime['ISBN'] = new_isbn

In [60]:
df_prime = df_prime.rename(columns={'Date':"first_appearance"})
df_prime.head()

,first_appearance,Book,Author,Year,ISBN,Author First,Author Last,ISBNx
0,January 4,The Godfather,Mario Puzo,1970,,Mario,Puzo,
1,February 8,The French Lieutenant's Woman,John Fowles,1970,0224616544,John,Fowles,0-224-61654-4
2,February 15,The Godfather,Mario Puzo,1970,,Mario,Puzo,
3,February 22,The French Lieutenant's Woman,John Fowles,1970,0224616544,John,Fowles,0-224-61654-4
4,May 10,Love Story,Erich Segal,1970,034012508X,Erich,Segal,0-340-12508-X


In [62]:
df_prime = df_prime.drop_duplicates(['Book'], keep='first')

In [91]:
df_prime.head()

,first_appearance,Book,Author,Year,ISBN
0,January 4,The Godfather,Mario Puzo,1970,
1,February 8,The French Lieutenant's Woman,John Fowles,1970,0224616544
4,May 10,Love Story,Erich Segal,1970,034012508X
6,February 21,QB VII,Leon Uris,1971,
7,April 25,The Passions of the Mind,Irving Stone,1971,0451134567


In [46]:
df_prime.to_csv("BestSellers with ISBN.csv")

In [ ]:
import sqlite3
conn = sqlite3.connect('Library.db')
c = conn.cursor()

In [ ]:
df_prime.to_sql("nyt_bestsellers", conn, if_exists="replace")

c.execute("SELECT * from nyt_bestsellers")
rows = c.fetchall()
 
for row in rows:
    print(row)

In [ ]:
results_df.to_sql("seattle_libcatalogue", conn, if_exists="replace")

c.execute("SELECT * from seattle_libcatalogue")
rows = c.fetchall()
 
for row in rows:
    print(row)